In [ ]:
!pip install anthropic

In [2]:
import anthropic
import pandas as pd
import math
import re
import time

In [3]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="",
)

In [ ]:
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=8192,
    messages=[
        {"role": "user", "content": "Based on the following real-life attack scenarios: 'During C0027, Scattered Spider impersonated legitimate IT personnel in phone calls and text messages either to direct victims to a credential harvesting site or getting victims to run commercial remote monitoring and management (RMM) tools. LAPSUS$ has called victims' help desk and impersonated legitimate users with previously gathered information in order to gain access to privileged accounts. During Operation Dream Job, Lazarus Group impersonated HR hiring personnel through LinkedIn messages and conducted interviews with victims in order to deceive them into downloading malware.', generate 97 additional procedure descriptions that closely relate to the MITRE technique ID T1656, technique name 'Impersonation', and tactic name 'Defense Evasion'. Please ensure that the new attack procedure descriptions emphasize the previous attack method specifically used for attack 'Impersonation' and the strategic objective 'Defense Evasion' used in these scenarios."
        }]
)

In [ ]:
print(message.content)

In [ ]:
print(message.content[0].text.split(":", 1)[1].strip())

In [ ]:
cleaned_content = message.content[0].text.split(":", 1)[1].strip()

In [34]:
df = pd.read_csv("/kaggle/input/traindata/train_data.csv")

In [35]:
df

,Technique,ID,Tactic-Name,Technique-Name,SubTechnique-Name,Platforms,Description,id
0,T1049,No sub-Techniques,Discovery,System Network Connections Discovery,No sub-Technique,"IaaS, Linux, Network, Windows, macOS",admin@338 actors used the following command fo...,T1049
1,T1049,No sub-Techniques,Discovery,System Network Connections Discovery,No sub-Technique,"IaaS, Linux, Network, Windows, macOS",Andariel has used the netstat -naop tcp comman...,T1049
2,T1049,No sub-Techniques,Discovery,System Network Connections Discovery,No sub-Technique,"IaaS, Linux, Network, Windows, macOS",APT3 has a tool that can enumerate current net...,T1049
3,T1049,No sub-Techniques,Discovery,System Network Connections Discovery,No sub-Technique,"IaaS, Linux, Network, Windows, macOS","APT38 installed a port monitoring tool, MAPMAK...",T1049
4,T1049,No sub-Techniques,Discovery,System Network Connections Discovery,No sub-Technique,"IaaS, Linux, Network, Windows, macOS",APT41 has enumerated IP addresses of network r...,T1049
...,...,...,...,...,...,...,...,...
8432,T1481,T1481.001,Command and Control,Web Service,Dead Drop Resolver,NaN,Anubis can retrieve the C2 address from Twitte...,T1481.001
8433,T1481,T1481.001,Command and Control,Web Service,Dead Drop Resolver,NaN,Red Alert 2.0 can fetch a backup C2 domain fro...,T1481.001
8434,T1481,T1481.001,Command and Control,Web Service,Dead Drop Resolver,NaN,XLoader for Android has fetched its C2 address...,T1481.001
8435,T1623,T1623.001,Execution,Command and Scripting Interpreter,Unix Shell,NaN,Device attestation can often detect jailbroken...,T1623.001


In [36]:
id_counts = df.groupby('id').size()

In [37]:
id_counts

id
T0813    1
T0815    2
T0826    3
T0828    1
T0829    2
        ..
T1656    3
T1657    1
T1659    2
T1661    2
T1663    2
Length: 537, dtype: int64

In [38]:
with open('id_counts_before.txt', 'w') as f:
    f.write(id_counts.to_string())

In [11]:
id_less_than_100 = id_counts[id_counts<100]

In [12]:
id_less_than_100

id
T0813    1
T0815    2
T0826    3
T0828    1
T0829    2
        ..
T1656    3
T1657    1
T1659    2
T1661    2
T1663    2
Length: 518, dtype: int64

In [13]:
id_list = id_less_than_100.index.tolist()

In [14]:
ids_per_file = 50

In [15]:
num_files = math.ceil(len(id_list) / ids_per_file)

In [16]:
num_files

11

In [17]:
id_chunks = [id_list[i:i + ids_per_file] for i in range(0, len(id_list), ids_per_file)]

In [18]:
for i, id_chunk in enumerate(id_chunks):
    # Filter the original DataFrame to include only rows with the current chunk of IDs
    df_chunk = df[df['id'].isin(id_chunk)]
    
    # Save the chunk to a new CSV file
    df_chunk.to_csv(f'ids_chunk_{i+1}.csv', index=False)

In [19]:
# List to store the number of unique IDs in each file
id_counts_per_file = []

# Number of files generated
num_files = 11  # For example, 5 files with 100 IDs and 1 file with 18 IDs

for i in range(1, num_files + 1):
    # Load each CSV file
    df_chunk = pd.read_csv(f'ids_chunk_{i}.csv')
    
    # Count the number of unique IDs in this chunk
    unique_ids = df_chunk['id'].nunique()
    
    # Append the count to the list
    id_counts_per_file.append(unique_ids)
    
    # Print the count for verification
    print(f'File ids_chunk_{i}.csv has {unique_ids} unique IDs.')

# Optional: Print the total number of IDs across all files
print(f'Total number of unique IDs across all files: {sum(id_counts_per_file)}')

File ids_chunk_1.csv has 50 unique IDs.
File ids_chunk_2.csv has 50 unique IDs.
File ids_chunk_3.csv has 50 unique IDs.
File ids_chunk_4.csv has 50 unique IDs.
File ids_chunk_5.csv has 50 unique IDs.
File ids_chunk_6.csv has 50 unique IDs.
File ids_chunk_7.csv has 50 unique IDs.
File ids_chunk_8.csv has 50 unique IDs.
File ids_chunk_9.csv has 50 unique IDs.
File ids_chunk_10.csv has 50 unique IDs.
File ids_chunk_11.csv has 18 unique IDs.
Total number of unique IDs across all files: 518


In [20]:
# Re-check the filtered list of IDs
filtered_id_counts = df.groupby('id').size()
ids_less_than_100 = filtered_id_counts[filtered_id_counts < 100]
id_list = ids_less_than_100.index.tolist()

print(f'Total IDs with less than 100 occurrences: {len(id_list)}')

Total IDs with less than 100 occurrences: 518


In [7]:
# Define the cleaning function
def clean_text(text):
    # First, split at the first colon and take the part after it
    try:
        text_after_colon = text.split(":", 1)[1].strip()
    except IndexError:
        text_after_colon = text.strip()
    
    # Split the text into lines
    lines = text_after_colon.split('\n')
    
    # Remove numbers followed by a dot and store only the sentences
    cleaned_sentences = []
    for line in lines:
        cleaned_line = re.sub(r'^\d+\.\s*', '', line)
        cleaned_sentences.append(cleaned_line)
    
    # Join the cleaned sentences back into a single text block or list of sentences
    cleaned_text = " ".join(cleaned_sentences).strip()
    
    return cleaned_text

In [22]:
ids_chunk_1 = pd.read_csv("/kaggle/working/ids_chunk_1.csv")
ids_chunk_2 = pd.read_csv("/kaggle/working/ids_chunk_2.csv")
ids_chunk_3 = pd.read_csv("/kaggle/working/ids_chunk_3.csv")
ids_chunk_4 = pd.read_csv("/kaggle/working/ids_chunk_4.csv")
ids_chunk_5 = pd.read_csv("/kaggle/working/ids_chunk_5.csv")
ids_chunk_6 = pd.read_csv("/kaggle/working/ids_chunk_6.csv")
ids_chunk_7 = pd.read_csv("/kaggle/working/ids_chunk_7.csv")
ids_chunk_8 = pd.read_csv("/kaggle/working/ids_chunk_8.csv")
ids_chunk_9 = pd.read_csv("/kaggle/working/ids_chunk_9.csv")
ids_chunk_10 = pd.read_csv("/kaggle/working/ids_chunk_10.csv")
ids_chunk_11 = pd.read_csv("/kaggle/working/ids_chunk_11.csv")

In [4]:
missing = pd.read_csv("/kaggle/input/missing-mitre-data/test_set_missing_ids.csv")

In [5]:
missing

,Technique,ID,Tactic-Name,Technique-Name,SubTechnique-Name,Platforms,Description,id
0,T0827,No sub-Techniques,Impact,Loss of Control,No sub-Technique,NaN,"Provide operators with redundant, out-of-band ...",T0827
1,T1647,No sub-Techniques,Defense Evasion,Plist File Modification,No sub-Technique,macOS,XCSSET uses the plutil command to modify the L...,T1647
2,T0837,No sub-Techniques,Impact,Loss of Protection,No sub-Technique,NaN,Industroyer contained a module which leveraged...,T0837
3,T0880,No sub-Techniques,Impact,Loss of Safety,No sub-Technique,NaN,Protection devices should have minimal digital...,T0880
4,T1658,No sub-Techniques,Execution,Exploitation for Client Execution,No sub-Technique,NaN,Security updates frequently contain patches to...,T1658
5,T1516,No sub-Techniques,"Defense Evasion, Impact",Input Injection,No sub-Technique,NaN,Users should be warned against granting access...,T1516
6,T1623,No sub-Techniques,Execution,Command and Scripting Interpreter,No sub-Technique,NaN,TianySpy can steal information via malicious J...,T1623
7,T1651,No sub-Techniques,Execution,Cloud Administration Command,No sub-Technique,"Azure AD, IaaS",APT29 has used Azure Run Command and Azure Adm...,T1651
8,T0880,No sub-Techniques,Impact,Loss of Safety,No sub-Technique,NaN,Ensure that all SIS are segmented from operati...,T0880
9,T1651,No sub-Techniques,Execution,Cloud Administration Command,No sub-Technique,"Azure AD, IaaS",Pacu can run commands on EC2 instances using A...,T1651


In [8]:
i=0
for id_value, group in missing.groupby('id'): #replace 'missing' with the name of the dataframe for example 'ids_chunk_1'
    current_count = len(group)
    if current_count < 100:
        i += 1
        n_descriptions = 100 - current_count
        print(f"{i}. Generating {n_descriptions} descriptions for Technique ID: {id_value}")
        
        # Combine all existing descriptions into a single text block
        existing_descriptions = " ".join(group['Description'].tolist())

        prompt_template = (
            f"Based on the following real-life attack scenarios: '{{existing_descriptions}}', "
            f"generate {{n_descriptions}} additional procedure descriptions that closely relate to the MITRE technique ID {id_value}, "
            f"technique name '{{technique_name}}', and "
            f"tactic name '{{tactic_name}}'. "
            f"Please ensure that the new attack procedure descriptions emphasize the previous attack method specifically used for attack '{{technique_name}}' and the strategic objective '{{tactic_name}}' used in these scenarios."
        )
        
        total_generated = 0
        max_retries = 10  # Maximum retries per technique ID
        all_generated_descriptions = []  # List to accumulate all generated descriptions
        
        while total_generated < n_descriptions and max_retries > 0:
            remaining_descriptions = n_descriptions - total_generated
            prompt = prompt_template.format(
                existing_descriptions=existing_descriptions,
                n_descriptions=remaining_descriptions,
                technique_name=group['Technique-Name'].iloc[0],
                tactic_name=group['Tactic-Name'].iloc[0]
            )
            
            # Model invocation (adjust this part according to your model API)
            message = client.messages.create(
                model="claude-3-5-sonnet-20240620",
                max_tokens=8192,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            
            # Clean the generated content
            generated_content = clean_text(message.content[0].text)
            
            # Split the cleaned content into individual descriptions
            new_generated_descriptions = generated_content.split('. ')[:remaining_descriptions]
            
            # Add new descriptions to the total list
            all_generated_descriptions.extend(new_generated_descriptions)
            total_generated = len(all_generated_descriptions)  # Update total generated count
            
            # Check if we got fewer than needed and decrement retry count
            if len(new_generated_descriptions) < remaining_descriptions:
                print(f"Warning: Not enough descriptions generated for Technique ID {id_value}. Expected: {remaining_descriptions}, Got: {len(new_generated_descriptions)}")
                max_retries -= 1  # Decrement retry count
                time.sleep(2)  # Brief delay before retry
            else:
                print(f"Successfully generated {len(new_generated_descriptions)} descriptions for Technique ID {id_value}.")
            
            # Delay to ensure we don't exceed 5 requests per minute
            time.sleep(12)  # Add a delay of 12 seconds to comply with rate limits

        # If retries were exhausted
        if total_generated < n_descriptions:
            print(f"Error: Could not generate the required number of descriptions for Technique ID {id_value} after multiple attempts.")

        # Create a DataFrame for the new descriptions if sufficient
        if total_generated > 0:
            new_descriptions_df = pd.DataFrame({
                'Technique': [id_value] * total_generated,
                'ID': [group['ID'].iloc[0]] * total_generated,
                'Tactic-Name': [group['Tactic-Name'].iloc[0]] * total_generated,
                'Technique-Name': [group['Technique-Name'].iloc[0]] * total_generated,
                'SubTechnique-Name': [group['SubTechnique-Name'].iloc[0]] * total_generated,
                'Platforms': [group['Platforms'].iloc[0]] * total_generated,
                'Description': all_generated_descriptions,  # Use the accumulated list
                'id': [id_value] * total_generated
            })
            
            # Append the new descriptions to the original DataFrame
            missing = pd.concat([missing, new_descriptions_df], ignore_index=True) #replace 'missing' with the name of the dataframe for example 'ids_chunk_1'

1. Generating 97 descriptions for Technique ID: T0827
Successfully generated 97 descriptions for Technique ID T0827.
2. Generating 99 descriptions for Technique ID: T0837
Successfully generated 99 descriptions for Technique ID T0837.
3. Generating 98 descriptions for Technique ID: T0880
Successfully generated 98 descriptions for Technique ID T0880.
4. Generating 99 descriptions for Technique ID: T1422
Successfully generated 99 descriptions for Technique ID T1422.
5. Generating 99 descriptions for Technique ID: T1471
Successfully generated 99 descriptions for Technique ID T1471.
6. Generating 99 descriptions for Technique ID: T1512
Successfully generated 99 descriptions for Technique ID T1512.
7. Generating 98 descriptions for Technique ID: T1516
Successfully generated 98 descriptions for Technique ID T1516.
8. Generating 99 descriptions for Technique ID: T1541
Successfully generated 99 descriptions for Technique ID T1541.
9. Generating 99 descriptions for Technique ID: T1582
Successful

In [9]:
# Save the augmented DataFrame back to the CSV
missing.to_csv('missing_augmented.csv', index=False)
print("Dataset augmentation complete.")

Dataset augmentation complete.


## Specific ID augmentation

In [ ]:
# Define the specific technique ID to focus on
specific_technique_id = 'T1553.004'  # Replace with the desired Technique ID

# Filter the DataFrame for the specific technique
group = new[new['id'] == specific_technique_id]

# Check if the count of descriptions for this technique is below the threshold (e.g., 100)
current_count = len(group)
if current_count < 100:
    n_descriptions = 100 - current_count
    print(f"Generating {n_descriptions} descriptions for Technique ID: {specific_technique_id}")
    
    # Combine all existing descriptions into a single text block
    existing_descriptions = " ".join(group['Description'].tolist())

    prompt_template = (
        f"Based on the following real-life attack scenarios: '{{existing_descriptions}}', "
        f"generate {{n_descriptions}} additional procedure descriptions that closely relate to the MITRE technique ID {specific_technique_id}, "
        f"technique name '{{technique_name}}', and "
        f"tactic name '{{tactic_name}}'. "
        f"Please ensure that the new attack procedure descriptions emphasize the previous attack method specifically used for attack '{{technique_name}}' and the strategic objective '{{tactic_name}}' used in these scenarios."
    )

    total_generated = 0
    max_retries = 5  # Maximum retries per technique ID
    all_generated_descriptions = []  # List to accumulate all generated descriptions
    
    while total_generated < n_descriptions and max_retries > 0:
        remaining_descriptions = n_descriptions - total_generated
        
        # Corrected formatting of the prompt
        prompt = prompt_template.format(
            existing_descriptions=existing_descriptions,
            n_descriptions=remaining_descriptions,
            technique_name=group['Technique-Name'].iloc[0],
            tactic_name=group['Tactic-Name'].iloc[0]
        )
        
        # Model invocation (adjust this part according to your model API)
        # Mocked response here as we can't call the API directly
        # Example: message = client.messages.create(...)
        
        # Simulate the cleaned generated content
        generated_content = "Generated description example. " * remaining_descriptions  # Mocked response
        generated_content = generated_content.strip()

        # Split the cleaned content into individual descriptions
        new_generated_descriptions = generated_content.split('. ')[:remaining_descriptions]
        
        # Add new descriptions to the total list
        all_generated_descriptions.extend(new_generated_descriptions)
        total_generated = len(all_generated_descriptions)  # Update total generated count
        
        # Check if we got fewer than needed and decrement retry count
        if len(new_generated_descriptions) < remaining_descriptions:
            print(f"Warning: Not enough descriptions generated for Technique ID {specific_technique_id}. Expected: {remaining_descriptions}, Got: {len(new_generated_descriptions)}")
            max_retries -= 1  # Decrement retry count
            # time.sleep(2)  # Brief delay before retry (Commented out for simulation)
        else:
            print(f"Successfully generated {len(new_generated_descriptions)} descriptions for Technique ID {specific_technique_id}.")
        
        # Simulated delay to ensure we don't exceed 5 requests per minute
        # time.sleep(12)  # Add a delay of 12 seconds to comply with rate limits (Commented out for simulation)

    # If retries were exhausted
    if total_generated < n_descriptions:
        print(f"Error: Could not generate the required number of descriptions for Technique ID {specific_technique_id} after multiple attempts.")

    # Create a DataFrame for the new descriptions if sufficient
    if total_generated > 0:
        new_descriptions_df = pd.DataFrame({
            'Technique': [group['Technique'].iloc[0]] * total_generated,
            'ID': [group['ID'].iloc[0]] * total_generated,
            'Tactic-Name': [group['Tactic-Name'].iloc[0]] * total_generated,
            'Technique-Name': [group['Technique-Name'].iloc[0]] * total_generated,
            'SubTechnique-Name': [group['SubTechnique-Name'].iloc[0]] * total_generated,
            'Platforms': [group['Platforms'].iloc[0]] * total_generated,
            'Description': all_generated_descriptions,  # Use the accumulated list
            'id': [specific_technique_id] * total_generated
        })
        
        # Append the new descriptions to the original DataFrame
        new = pd.concat([new, new_descriptions_df], ignore_index=True)
else:
    print(f"Technique ID {specific_technique_id} already has {current_count} descriptions, no additional generation needed.")


In [ ]:
new.to_csv('ids_chunk_7_augmented.csv', index=False)
print("Dataset augmentation complete.")

In [27]:
# Filter IDs with counts more than or equal to 100
id_more_than_100 = id_counts[id_counts >= 100]

# Get the list of IDs that have more than or equal to 100 occurrences
id_list_more_than_100 = id_more_than_100.index.tolist()

# Filter the original DataFrame to include only rows with these IDs
df_more_than_100 = df[df['id'].isin(id_list_more_than_100)]

# Save this filtered DataFrame to a new CSV file
df_more_than_100.to_csv('ids_more_than_100.csv', index=False)